In [1]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv
import shutil
import gc

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
subs = ['asu11','asu12']

In [3]:
#get cal scans
for sub in subs:
    for run in range(1,5):
        cal_dir = home_dir + '/data/' + sub +'/cal/'
        pe1s = glob.glob(cal_dir + 'cal*')

        #pe1 from cal
        if run < 3:
            pe1 = pe1s[0] #take the first one
        else:
            pe1 = pe1s[1]
            
        out_pe1 = cal_dir + 'pe1_slice_' + str(run)
        cmd = ['fslroi',pe1,out_pe1,'4','1'] #last slice of pe
        os.system(' '.join(cmd))

        #pe0 from main scans
        out_pe0 = cal_dir + 'pe0_slice_' + str(run)
        in_f =  home_dir + '/data/' + sub +'/func/run' + str(run) + '.nii.gz'
        data = nib.load(in_f)
        idx = data.shape[-1]//2 #middle volume
        cmd = ['fslroi',in_f,out_pe0,str(idx),'1']
        os.system(' '.join(cmd))

        #merge
        out_f = cal_dir + 'merged_' + str(run)
        cmd = ['fslmerge','-t',out_f,out_pe0,out_pe1]
        os.system(' '.join(cmd))

        #mccflirt
        cmd = ['mcflirt','-in',out_f,'-refvol','0'] #register to functional run
        os.system(' '.join(cmd))
